# Adaptive Optics Simulation using PyTomo

This notebook demonstrates how to build a tomographic reconstructor in phase space using PyTomo. It includes:
- Loading configuration parameters from an INI file
- Creating an AO system
- Constructing a reconstructor using the tomoReconstructor class
- Updating the modified filtered subaperture mask and recalculating the reconstructor

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import tomoAO
tomoAO.__version__

In [ ]:
ao_mode = "MLAO"

config_dir = "/home/joaomonteiro/Desktop/test_pytomo/"
config_file = "config.ini"

# Loading the config
config_vars = tomoAO.IO.load_from_ini(config_file, ao_mode=ao_mode, config_dir=config_dir)


## Creating the Adaptive Optics System
Using the loaded configuration, we instantiate an AO system

In [ ]:
aoSys = tomoAO.Simulation.AOSystem(config_vars)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

titles = ["Unfiltered Subapertures Mask", "Filtered Subapertures Mask"]
matrices = [aoSys.unfiltered_subap_mask, aoSys.filtered_subap_mask]

for ax, matrix, title in zip(axes.flat, matrices, titles):
    im = ax.imshow(matrix)
    ax.set_title(title)
    fig.colorbar(im, ax=ax)

## Building the Tomographic Reconstructor
We use the tomoReconstructor class to construct a reconstructor from scratch

In [ ]:
from tomoAO.Reconstruction.reconClassType import tomoReconstructor

In [ ]:
rec = tomoReconstructor(aoSys=aoSys, alpha=10, os=2)

In [ ]:
plt.imshow(rec.R_unfiltered)
plt.colorbar()
plt.show()

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(10, 10))

titles = ["Cox", "Cxx", "Fitting Matrix", "Reconstructor"]
matrices = [rec.Cox[0], rec.Cxx, rec.fittingMatrix, rec.Reconstructor[0]]

for ax, matrix, title in zip(axes.flat, matrices, titles):
    im = ax.imshow(matrix, cmap="viridis", aspect="auto")
    ax.set_title(title)
    fig.colorbar(im, ax=ax)

## Updating and Displaying the Modified Filtered Subaperture Mask
We modify the existing mask, recompute the reconstructor, and visualize the changes


In [ ]:
# Modify the filtered subaperture mask and recompute the reconstructor
# Example: Transpose the existing mask to introduce a different configuration
rec.filtered_subap_mask = aoSys.filtered_subap_mask.T
plt.imshow(rec.filtered_subap_mask)

In [ ]:
# Computing the new weight vector
# The weight vector can be user-defined by assigning a custom vector to `rec.weight_vector`
rec.computeDefaultWeightVector()

In [ ]:
#Computing the new noise covariance matrix
# The reconstructor noise covariance matrix can be user-defined by assigning a custom vector to `rec.noise_covariance_matrix`
rec.computeDefaultNoiseCovariance()

In [ ]:
rec.buildReconstructor()

# Building the reconstructor from function

In [ ]:
from tomoAO.Reconstruction import tomographic_reconstructor_phase_space

### If the debug variable is not specified or is set to False, only the final reconstructor will be returned.


In [ ]:
rec_fn = tomographic_reconstructor_phase_space(config_file=config_file,config_dir=config_dir,
                                               alpha=10)

### If the debug variable is set to True, the entire object will be returned.


In [ ]:
rec_fn_debug = tomographic_reconstructor_phase_space(config_file=config_file,config_dir=config_dir,
                                                     alpha=10, debug=True)
